In [1]:
import sympy as sym
import symnum
import symnum.numpy as snp
import numpy as np
import multiprocessing

In [2]:
y = snp.array([1, 0])
σ = 1

def forward_func(x):
    return snp.array([[1, -1], [-2, 2]]) @ snp.array(
        [snp.cos(-x[1]**2 + 3 * x[0]), snp.sin(x[0] - 1)])

def neg_log_dens(x):
    return (
        snp.sum((y - forward_func(x))**2 / σ**2) + 
        snp.sum(x**2)) / 2

## Symbolic evaluation and derivatives

In [3]:
x = symnum.named_array('x', 2)

In [4]:
forward_func(x)

[-sin(x[0] - 1) + cos(3*x[0] - x[1]**2), 2*sin(x[0] - 1) - 2*cos(3*x[0] - x[1]**2)]

In [5]:
symnum.sympy_jacobian(forward_func)(x)

[[-3*sin(3*x[0] - x[1]**2) - cos(x[0] - 1), 2*x[1]*sin(3*x[0] - x[1]**2)], [6*sin(3*x[0] - x[1]**2) + 2*cos(x[0] - 1), -4*x[1]*sin(3*x[0] - x[1]**2)]]

In [6]:
neg_log_dens(x)

x[0]**2/2 + x[1]**2/2 + (-2*sin(x[0] - 1) + 2*cos(3*x[0] - x[1]**2))**2/2 + (sin(x[0] - 1) - cos(3*x[0] - x[1]**2) + 1)**2/2

In [7]:
symnum.sympy_grad(neg_log_dens)(x)

[x[0] + (-2*sin(x[0] - 1) + 2*cos(3*x[0] - x[1]**2))*(-12*sin(3*x[0] - x[1]**2) - 4*cos(x[0] - 1))/2 + (6*sin(3*x[0] - x[1]**2) + 2*cos(x[0] - 1))*(sin(x[0] - 1) - cos(3*x[0] - x[1]**2) + 1)/2, 4*x[1]*(-2*sin(x[0] - 1) + 2*cos(3*x[0] - x[1]**2))*sin(3*x[0] - x[1]**2) - 2*x[1]*(sin(x[0] - 1) - cos(3*x[0] - x[1]**2) + 1)*sin(3*x[0] - x[1]**2) + x[1]]

In [8]:
symnum.sympy_hessian(neg_log_dens)(x)

[[(-2*sin(x[0] - 1) + 2*cos(3*x[0] - x[1]**2))*(2*sin(x[0] - 1) - 18*cos(3*x[0] - x[1]**2)) + (-sin(x[0] - 1) + 9*cos(3*x[0] - x[1]**2))*(sin(x[0] - 1) - cos(3*x[0] - x[1]**2) + 1) + (-6*sin(3*x[0] - x[1]**2) - 2*cos(x[0] - 1))**2 + (3*sin(3*x[0] - x[1]**2) + cos(x[0] - 1))**2 + 1, 12*x[1]*(-2*sin(x[0] - 1) + 2*cos(3*x[0] - x[1]**2))*cos(3*x[0] - x[1]**2) + 4*x[1]*(-6*sin(3*x[0] - x[1]**2) - 2*cos(x[0] - 1))*sin(3*x[0] - x[1]**2) - 2*x[1]*(3*sin(3*x[0] - x[1]**2) + cos(x[0] - 1))*sin(3*x[0] - x[1]**2) - 6*x[1]*(sin(x[0] - 1) - cos(3*x[0] - x[1]**2) + 1)*cos(3*x[0] - x[1]**2)], [12*x[1]*(-2*sin(x[0] - 1) + 2*cos(3*x[0] - x[1]**2))*cos(3*x[0] - x[1]**2) + 4*x[1]*(-6*sin(3*x[0] - x[1]**2) - 2*cos(x[0] - 1))*sin(3*x[0] - x[1]**2) - 2*x[1]*(3*sin(3*x[0] - x[1]**2) + cos(x[0] - 1))*sin(3*x[0] - x[1]**2) - 6*x[1]*(sin(x[0] - 1) - cos(3*x[0] - x[1]**2) + 1)*cos(3*x[0] - x[1]**2), -8*x[1]**2*(-2*sin(x[0] - 1) + 2*cos(3*x[0] - x[1]**2))*cos(3*x[0] - x[1]**2) + 4*x[1]**2*(sin(x[0] - 1) - cos(3*x[

## NumPy function generation

In [9]:
x_np = np.array([0.2, 1.1])

In [10]:
symnum.numpify(2)(forward_func)(x_np)

array([ 1.53700411, -3.07400822])

In [11]:
symnum.numpify(2)(neg_log_dens)(x_np)

5.493949967002545

In [12]:
grad_neg_log_dens = symnum.numpy_grad(neg_log_dens, 2)
grad_neg_log_dens(x_np)

array([ 7.03139355, -7.32518763])

In [13]:
hess_neg_log_dens = symnum.numpy_hessian(neg_log_dens, 2)
hess_neg_log_dens(x_np)

array([[-47.88846093,  29.72428278],
       [ 29.72428278, -25.23749599]])

In [14]:
mtp_neg_log_dens = symnum.numpy_mtp(neg_log_dens, 2)
mtp_neg_log_dens(x_np)(hess_neg_log_dens(x_np))

array([ 24450.22234805, -17321.69277479])

In [15]:
jacob_forward_func = symnum.numpy_jacobian(forward_func, 2)
jacob_forward_func(x_np)

array([[ 1.02189567, -1.26030841],
       [-2.04379134,  2.52061682]])

In [16]:
mhp_forward_func = symnum.numpy_mhp(forward_func, 2)
mhp_forward_func(x_np)(jacob_forward_func(x_np))

array([-75.4463863 ,  59.85934877])

## Compatibility with `multiprocessing`

In [17]:
pool = multiprocessing.Pool(4)

In [18]:
pool.map(grad_neg_log_dens, [x_np + 0.1 * i for i in range(8)])

[array([ 7.03139355, -7.32518763]),
 array([ 5.36166701, -6.83240645]),
 array([ 4.05701734, -6.32916535]),
 array([ 3.1698736 , -5.90591748]),
 array([ 2.69164583, -5.61080858]),
 array([ 2.56667444, -5.4433092 ]),
 array([ 2.70213256, -5.34858144]),
 array([ 2.97340715, -5.21063384])]